In [2]:
import json
import urllib2 
import os
import sys
from pprint import pprint
import pandas as pd
from datetime import datetime
import psycopg2

In [ ]:
# http://bustime.mta.info/wiki/Developers/SIRIMonitoredVehicleJourney

In [3]:
keys = json.load(open("./credentials.json"))
apikey = keys['MTA'] # MTA API Key

try:
    url = "http://bustime.mta.info/api/siri/vehicle-monitoring.json?key=%s\
&version=2&VehicleMonitoringDetailLevel=calls"%(apikey)

    response = urllib2.urlopen(url)  
except urllib2.HTTPError:
    print("Incorrect MTA Key")
else:

    data = response.read().decode("utf-8")
    info = json.loads(data)

bus_list = []
vehicles = (info["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"][0]["VehicleActivity"])

for i in range(len(vehicles)):
    bus_data = {}
    loc = vehicles[i]["MonitoredVehicleJourney"]["VehicleLocation"]

    bus_data['datetime'] = vehicles[i]['RecordedAtTime']
    bus_data['device_id'] = vehicles[i]["MonitoredVehicleJourney"]['VehicleRef']
    bus_data['lat'] = loc['Latitude'] 
    bus_data['lon'] = loc['Longitude']
    bus_data['route_id'] = (vehicles[i]["MonitoredVehicleJourney"]['PublishedLineName'][0] + "_" +
                            vehicles[i]["MonitoredVehicleJourney"]['DirectionRef'])
#     bus_data['bearing'] = vehicles[i]["MonitoredVehicleJourney"]['Bearing']
#     bus_data['route'] = vehicles[i]["MonitoredVehicleJourney"]['PublishedLineName'][0]
    bus_list.append(bus_data)

bus_df = pd.DataFrame(bus_list)
bus_df.datetime = pd.to_datetime(bus_df.datetime, utc=True)

In [4]:
bus_df.head(2)

,datetime,device_id,lat,lon,route_id
0,2019-09-29 03:51:53,MTA NYCT_3824,40.731774,-73.995226,M55_1
1,2019-09-29 03:51:48,MTA NYCT_3826,40.762958,-73.989647,M50_1


In [5]:
keys = json.load(open("./credentials.json"))

host = keys['postgres'][0]['host']
database = keys['postgres'][0]['database']
user = keys['postgres'][0]['user']
password = keys['postgres'][0]['password']

con = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = con.cursor()

cur.execute('TRUNCATE bus.device_test;')
con.commit()

In [6]:
query = 'insert into bus.device_test (datetime, device_id, lat, lon, route_id) VALUES (%s, %s, %s, %s, %s);'
for idx, row in bus_df.iterrows():
    cur.execute(query,
                (datetime.strptime(str(row['datetime']).split(".")[0], '%Y-%m-%d %H:%M:%S'), 
                 row['device_id'],
                 row['lat'], row['lon'],
                 row['route_id']))
con.commit()
# con.close()

cur.execute('''update bus.device_test set geom=st_SetSrid(st_MakePoint(cast(lon as float), cast(lat as float)), 4326);''')
con.commit()

In [7]:
query = "SELECT * FROM bus.device_test WHERE datetime >= now()::timestamp - interval '10 days' LIMIT 10;"

In [8]:
pd.read_sql(sql=query, con=con)

,lon,lat,device_id,datetime,geom,route_id
0,-73.995226,40.731774,MTA NYCT_3824,2019-09-29 03:51:53,0101000020E6100000408864C8B17F52C014083BC5AA5D...,M55_1
1,-73.989647,40.762958,MTA NYCT_3826,2019-09-29 03:51:48,0101000020E610000069E55E60567F52C05C1C959BA861...,M50_1
2,-73.996178,40.73826,MTA NYCT_3829,2019-09-29 03:52:07,0101000020E6100000A9BF5E61C17F52C0F4F8BD4D7F5E...,M7_0
3,-74.014267,40.704265,MTA NYCT_3811,2019-09-29 03:51:57,0101000020E61000005F9A22C0E98052C0D55B035B255A...,M55_0
4,-73.972537,40.749927,MTA NYCT_3815,2019-09-29 03:52:05,0101000020E61000009949D40B3E7E52C096B1A19BFD5F...,M42_1
5,-73.975711,40.718899,MTA NYCT_6095,2019-09-29 03:52:11,0101000020E61000003DD68C0C727E52C04510E7E1045C...,M14D-SBS_0
6,-74.008295,40.744433,MTA NYCT_6098,2019-09-29 03:51:50,0101000020E61000001A6EC0E7878052C016889E94495F...,M14D-SBS_1
7,-73.974922,40.724765,MTA NYCT_3804,2019-09-29 03:51:50,0101000020E6100000A9893E1F657E52C089247A19C55C...,M8_0
8,-73.975994,40.763809,MTA NYCT_3809,2019-09-29 03:52:10,0101000020E6100000E7C589AF767E52C0FBB1497EC461...,M57_1
9,-73.998268,40.760791,MTA NYCT_3808,2019-09-29 03:51:52,0101000020E61000002C29779FE37F52C0AA0B78996161...,M42_1


In [33]:
info['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][0]['MonitoredVehicleJourney'].keys()

[u'OriginRef',
 u'Bearing',
 u'VehicleLocation',
 u'SituationRef',
 u'OperatorRef',
 u'VehicleRef',
 u'BlockRef',
 u'DestinationName',
 u'DestinationRef',
 u'JourneyPatternRef',
 u'MonitoredCall',
 u'LineRef',
 u'FramedVehicleJourneyRef',
 u'PublishedLineName',
 u'Monitored',
 u'OnwardCalls',
 u'DirectionRef',
 u'ProgressRate']

[u'OriginRef', - Not required
 u'Bearing', - Vehicle bearing: 0 is East, increments counter-clockwise (May be used)
 u'VehicleLocation', - Use this for location
 u'SituationRef', - Not required for demo
 u'OperatorRef', - - Not required
 u'VehicleRef',- use as device_id
 u'BlockRef',- Not required
 u'DestinationName',- Not required for demo (may be used)
 u'DestinationRef',- Not required
 u'JourneyPatternRef', - Not required for demo
 u'MonitoredCall', -  - Not required for demo
 u'LineRef', - Not required for demo
 u'FramedVehicleJourneyRef', - Not required for demo
 u'PublishedLineName', - use for bus id 
 u'Monitored',- Not required for demo
 u'OnwardCalls',- Not required for demo
 u'DirectionRef', - route_id finding
 u'ProgressRate']- Not required for demo

# Future use- show nearest bus stops

In [7]:
lat = 40.680875500000006
lon = -73.94772689999999

In [9]:
try:
    url = "http://bustime.mta.info/api/where/stops-for-location.json?\
lat={0}&lon={1}&latSpan=0.005&lonSpan=0.005&key={2}".format(lat, lon, apikey)

    response = urllib2.urlopen(url)  
except urllib2.HTTPError:
    print("Incorrect MTA Key")
else:

    data = response.read().decode("utf-8")
    info = json.loads(data)

In [10]:
info

{u'code': 200,
 u'currentTime': 1569717035417L,
 u'data': {u'limitExceeded': False,
  u'stops': [{u'code': u'302380',
    u'direction': u'E',
    u'id': u'MTA_302380',
    u'lat': 40.680301,
    u'locationType': 0,
    u'lon': -73.949161,
    u'name': u'FULTON ST/NOSTRAND AV',
    u'routes': [{u'agency': {u'disclaimer': u'',
       u'id': u'MTA NYCT',
       u'lang': u'en',
       u'name': u'MTA New York City Transit',
       u'phone': u'718-330-1234',
       u'privateService': False,
       u'timezone': u'America/New_York',
       u'url': u'http://www.mta.info'},
      u'color': u'6CBE45',
      u'description': u'via Fulton St',
      u'id': u'MTA NYCT_B25',
      u'longName': u'Dwntn Bklyn & DUMBO - Broadway Junction',
      u'shortName': u'B25',
      u'textColor': u'FFFFFF',
      u'type': 3,
      u'url': u''}],
    u'wheelchairBoarding': u'UNKNOWN'},
   {u'code': u'302381',
    u'direction': u'E',
    u'id': u'MTA_302381',
    u'lat': 40.680116,
    u'locationType': 0,
    u'lon'